# Simple notebook for accurate predictions using the build-in Inception-ResNet V2 module in TensorFlow (tf.keras).

I got an embarrasing score back when this competition was running so I decided to revisit with this notebook. :-)

With validation turned off (validation_split set to 0.0 in train_datagen) this scores 97% as "late submission".

**First import TensorFlow and Pandas**

In [2]:
import tensorflow as tf
import pandas as pd

**Secondly define the neural network. We use the build-in module for Inception-ResNet V2 in tf.keras.**

We don't import the output layer, however as we don't have 1000 different categories. Instead we add a few layers for our 12 catagories.

In [3]:
def define_model(width, height):
    model_input = tf.keras.layers.Input(shape=(width, height, 3), name='image_input')
    model_main = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet')(model_input)
    model_dense1 = tf.keras.layers.Flatten()(model_main)
    model_dense2 = tf.keras.layers.Dense(128, activation='relu')(model_dense1)
    model_out = tf.keras.layers.Dense(5, activation="softmax")(model_dense2)

    model = tf.keras.models.Model(model_input,  model_out)
    optimizer = tf.keras.optimizers.Adam(lr=0.00004, beta_1=0.9, beta_2=0.999)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

**Next, the code for the data generators that take care of traversing through the directories with images and augmenting the images as needed for training.**

In [4]:
def define_generators():
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=360,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.5,
        vertical_flip=True,
        horizontal_flip=True,
        validation_split=0.0, # change to use validation instead of training on entire training set
    )

    train_generator = train_datagen.flow_from_directory(
        directory='/kaggle/input/task-2/dataset-for-task2/dataset-for-task2/train',
        target_size=(width, height),
        batch_size=batch_size,
        color_mode='rgb',
        class_mode="categorical",
        subset='training',
    )

    validation_generator = train_datagen.flow_from_directory(
        directory='/kaggle/input/task-2/dataset-for-task2/dataset-for-task2/train',
        target_size=(width, height),
        batch_size=batch_size,
        color_mode='rgb',
        class_mode="categorical",
        subset='validation',
    )

    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

    test_generator = test_datagen.flow_from_directory(
        directory='/kaggle/input/task-2/dataset-for-task2/dataset-for-task2',
        classes=['test'],
        target_size=(width, height),
        batch_size=1,
        color_mode='rgb',
        shuffle=False,
        class_mode='categorical')

    return train_generator, validation_generator, test_generator

**Define the checkpoint save callback on validation accuracy.**

It is not currently used, but you can if you want to work on the model with the highest accuracy instead of the last training epoch.

In [5]:
def define_callbacks():
    save_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor='val_acc',
        save_best_only=True,
        verbose=1
    )

    return save_callback

**Define some constants.**

These are number of epochs to train, images per batch, image width and height in pixels and the names of each class/species for the submission file

In [6]:
nb_epoch     = 40
batch_size   = 16
width        = 299
height       = 299
species_list = ["Black-grass",  "Common wheat",
                "Loose Silky-bent", "Scentless Mayweed", 
                "Sugar beet"]
# pecies_list = ["Black-grass", "Charlock", "Cleavers", "Common Chickweed", "Common wheat", "Fat Hen",
#                 "Loose Silky-bent", "Maize", "Scentless Mayweed", "Shepherds Purse", "Small-flowered Cranesbill",
#                 "Sugar beet"]

**Now, define the model and fit it with the training data.**

In [9]:
model = define_model(width, height)
model.summary()
train_generator, validation_generator, test_generator = define_generators()
save_callback = define_callbacks()

model.fit(
    train_generator,
    epochs=nb_epoch,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data= validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[save_callback] #UNCOMMENT THIS LINE TO SAVE BEST VAL_ACC MODEL
)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 299, 299, 3)]     0         
_________________________________________________________________
inception_resnet_v2 (Functio (None, None, None, 1536)  54336736  
_________________________________________________________________
flatten_1 (Flatten)          (None, 98304)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               12583040  
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645       
Total params: 66,920,421
Trainable params: 66,859,877
Non-trainable params: 60,544
_________________________________________________________________
Found 500 images belonging to 5 classes.
Found 0 images belonging to 5 classes.
Found 378 images belonging t

**Cool, now we (hopefully) have a model that can predict the species!**

Call it to get the predictions, and create a pandas dataframe with the species names of the highest probabilities. finally save the dataframe as the submission file.

In [11]:
pd.set_option('display.max_rows', None)  # None 表示无限制

In [15]:
predictions = model.predict(test_generator, steps=test_generator.samples)

class_list = []

for i in range(0, predictions.shape[0]):
  y_class = predictions[i, :].argmax(axis=-1)
  class_list += [species_list[y_class]]

submission = pd.DataFrame()
submission['file'] = test_generator.filenames
submission['file'] = submission['file'].str.replace(r'test/', '')
submission['species'] = class_list
print(submission[['species']])
submission.to_csv('/kaggle/working/submission.csv', index=False)

print('Submission file generated. All done.')

               file            species
0    abkieurfrr.png         Sugar beet
1    ablzkpojla.png         Sugar beet
2    acmbgjflct.png  Scentless Mayweed
3    afhcnytjtd.png   Loose Silky-bent
4    ahoyeyvosg.png   Loose Silky-bent
5    aijmdaocym.png        Black-grass
6    ajjwrhipgh.png   Loose Silky-bent
7    apjrnavrnr.png         Sugar beet
8    arkzcnxddq.png  Scentless Mayweed
9    armpwctjip.png         Sugar beet
10   asgkdhflws.png   Loose Silky-bent
11   ashctxuyff.png         Sugar beet
12   aupdlpczgg.png        Black-grass
13   axckzqattx.png       Common wheat
14   axzjvqaozv.png        Black-grass
15   azlgnuwzcc.png   Loose Silky-bent
16   bbtaboqkyy.png         Sugar beet
17   bcftvziwoc.png  Scentless Mayweed
18   bcgxgybctb.png  Scentless Mayweed
19   bddyvdvkgo.png   Loose Silky-bent
20   behrxwzueb.png        Black-grass
21   bfgtphhmiu.png        Black-grass
22   biwpjwytdz.png   Loose Silky-bent
23   bjogjvvgoo.png        Black-grass
24   bodktpofui.png      

In [16]:
submission

,file,species
0,abkieurfrr.png,Sugar beet
1,ablzkpojla.png,Sugar beet
2,acmbgjflct.png,Scentless Mayweed
3,afhcnytjtd.png,Loose Silky-bent
4,ahoyeyvosg.png,Loose Silky-bent
5,aijmdaocym.png,Black-grass
6,ajjwrhipgh.png,Loose Silky-bent
7,apjrnavrnr.png,Sugar beet
8,arkzcnxddq.png,Scentless Mayweed
9,armpwctjip.png,Sugar beet
